In [1]:
!pip install librosa
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 50.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 33.9 MB/s eta 0:00:0000:0100:01
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23702 sha256=81eb647258dc976467e642955463428b352d39be74b47acceeeedf94f292d2f1
  Stored in directory: /root/.cache/pip/wheels/e4/76/a4/cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.4 MB/s

In [2]:
########################################################################
# import default libraries
########################################################################
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import
from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
import common as com
import keras_model
import pandas as pd
import keras
import librosa
import tensorflow as tf
import librosa.display
import tensorflow_io as tfio

############

In [3]:
############################################################


########################################################################
# load parameter.yaml
########################################################################
param = com.yaml_load()
########################################################################
saved_weight = os.path.join(param["P_MODELSAVE"], 'dataweights.{epoch:02d}-{val_acc:.2f}.hdf5')

modelchk = keras.callbacks.ModelCheckpoint(saved_weight, 
                                      monitor='val_acc', 
                                      verbose=1,
                                      save_best_only=True, 
                                      save_weights_only=False,
                                      mode='auto',
                                      period=2)

tensorboard = keras.callbacks.TensorBoard(log_dir=param["P_LOGS"],
                                          histogram_freq=0,
                                          write_graph=True,
                                          write_images=True)

csv_logger = keras.callbacks.CSVLogger(f'{param["P_LOGS"]}/keras_log.csv',
                                       append=True)

# model_unet = keras_model.get_model(input_shape=(1,param["feature"]["n_mels"],param["feature"]["n_frames"]), lr = param["fit"]["lr"])

########################################################################
# visualizer
########################################################################
class visualizer(object):
    def __init__(self):
        import matplotlib.pyplot as plt
        self.plt = plt
        self.fig = self.plt.figure(figsize=(7, 5))
        self.plt.subplots_adjust(wspace=0.3, hspace=0.3)

    def loss_plot(self, loss, val_loss):
        """
        Plot loss curve.

        loss : list [ float ]
            training loss time series.
        val_loss : list [ float ]
            validation loss time series.

        return   : None
        """
        ax = self.fig.add_subplot(1, 1, 1)
        ax.cla()
        ax.plot(loss)
        ax.plot(val_loss)
        ax.set_title("Model loss")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(["Train", "Validation"], loc="upper right")

    def save_figure(self, name):
        """
        Save figure.

        name : str
            save png file path.

        return : None
        """
        self.plt.savefig(name)


########################################################################


########################################################################
# get data from the list for file paths
########################################################################
def file_to_vectors(file_name,
                    n_mels=128,
                    n_fft=1024,
                    hop_length=431,
                    power=2.0):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """

    # generate melspectrogram using librosa
    file_contents = tf.io.read_file(file_name)
    y, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    y = tf.squeeze(y, axis=[-1])
    spectrogram = tfio.audio.spectrogram(y, nfft=n_fft, window=n_fft, stride=hop_length)
    mel_spectrogram = tfio.audio.melscale(spectrogram, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
    dbscale_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=120)
    masked_mel_spectrogram = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=30)
    masked_mel_spectrogram = tfio.audio.freq_mask(masked_mel_spectrogram, param=20)
    dbscale_mel_spectrogram = tf.expand_dims(dbscale_mel_spectrogram, axis=2)
    masked_mel_spectrogram = tf.expand_dims(masked_mel_spectrogram, axis=2)
    # convert melspectrogram to log mel energies
    return masked_mel_spectrogram, dbscale_mel_spectrogram

In [6]:

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car1"
# loop of the base directory
print("\n===========================")

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")



============== DATASET_GENERATOR ==============


In [7]:
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat(100).map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat(100).map(file_to_vectors).batch(32).prefetch(1)

In [8]:
# set path
machine_type = os.path.split(target_dir)[1]
model_file_path = "{model}/model_{machine_type}.hdf5".format(model=param["model_directory"],
                                                                machine_type=machine_type)

history_img = "{model}/history_{machine_type}.png".format(model=param["model_directory"],
                                                            machine_type=machine_type)
# pickle file for storing anomaly score distribution
score_distr_file_path = "{model}/score_distr_{machine_type}.pkl".format(model=param["model_directory"],
                                                                        machine_type=machine_type)

In [6]:
# number of vectors for each wave file
# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_3((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset)


pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)


# # calculate y_pred for fitting anomaly score distribution
# y_pred = []
# start_idx = 0
# for file_idx in range(len(files)):
#         y_pred.append(np.mean(np.square(data[file_idx,: ,  :] 
#                                 - model.predict(data[file_idx,: , :]))))
#         start_idx += n_vectors_ea_file

# # fit anomaly score distribution
# shape_hat, loc_hat, scale_hat = scipy.stats.gamma.fit(y_pred)
# gamma_params = [shape_hat, loc_hat, scale_hat]
# joblib.dump(gamma_params, score_distr_file_path)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
visualizer.save_figure(history_img)

print("============== END TRAINING ==============")


gc.collect()


============== MODEL TRAINING ==============
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']           
 ation)                         )                

KeyboardInterrupt: 

In [9]:

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car1"
# loop of the base directory
print("\n===========================")

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)


pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']           


2022-10-19 02:23:02,314 - INFO - save_model -> /notebooks/model/model_car1.hdf5


============== END TRAINING ==============


0

In [4]:
# del train_dataset
# del pos
# del val_dataset
# del model
machine_type=2
# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car2"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car2.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']           


2022-10-20 01:04:05,226 - INFO - save_model -> /notebooks/model/model_car2.hdf5


============== END TRAINING ==============


4

In [5]:

del train_dataset
del pos
del val_dataset
del model
# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car3"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car3.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)


machine_type = 'car3'
pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 01:44:53,441 - INFO - save_model -> /notebooks/model/model_car3.hdf5


============== END TRAINING ==============


14878

In [6]:
del train_dataset
del pos
del val_dataset
del model

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car4"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car4.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)


machine_type = 'car4'

pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 02:25:45,852 - INFO - save_model -> /notebooks/model/model_car4.hdf5


============== END TRAINING ==============


14878

In [7]:
del train_dataset
del pos
del val_dataset
del model
machine_type = 'car5'

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car5"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car5.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 03:06:14,219 - INFO - save_model -> /notebooks/model/model_car5.hdf5


============== END TRAINING ==============


14878

In [8]:
del train_dataset
del pos
del val_dataset
del model
machine_type = 'car6'

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car6"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car6.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()
history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 03:48:44,073 - INFO - save_model -> /notebooks/model/model_car6.hdf5


============== END TRAINING ==============


14878

In [9]:
del train_dataset
del pos
del val_dataset
del model
machine_type = 'car7'

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car7"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car7.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 04:31:12,239 - INFO - save_model -> /notebooks/model/model_car7.hdf5


============== END TRAINING ==============


14878

In [10]:
del train_dataset
del pos
del val_dataset
del model
machine_type = 'car8'

# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car8"
# loop of the base directory
print("\n===========================")
model_file_path = "{model}/model_car8.hdf5".format(model=param["model_directory"])

# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training


pos = tf.data.Dataset.list_files(f"{target_dir}/train/*.wav")
train_size = int(0.9 * pos.cardinality().numpy())
val_size = int(0.1 * pos.cardinality().numpy())
train_dataset = pos.shuffle(buffer_size=10000).take(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)
val_dataset = pos.shuffle(buffer_size=10000).skip(train_size).repeat().map(file_to_vectors).batch(32).prefetch(1)

# train model

print("============== MODEL TRAINING ==============")
model = keras_model.get_model_4((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=model_opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.summary()

history = model.fit(train_dataset,
                    shuffle=param["fit"]["shuffle"],
                    verbose=param["fit"]["verbose"],
                    validation_data = val_dataset,
                   epochs = 30,
                   steps_per_epoch=80,
                   validation_steps=2)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
# visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
# visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

gc.collect()



============== DATASET_GENERATOR ==============
============== MODEL TRAINING ==============
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 512, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 512, 128, 16  160         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 512, 128, 16  64         ['block_1_conv[0][0]']          

2022-10-20 05:13:35,961 - INFO - save_model -> /notebooks/model/model_car8.hdf5


============== END TRAINING ==============


14878